In [1]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
from os import listdir

In [2]:
import cufflinks as cf
cf.go_offline()

In [6]:
# list of Pickle files names
list_dir = listdir(Path.cwd() / 'Pickles' / 'K_one_w')
files = [ld for ld in list_dir if ld[-2:-1].isdigit()]

In [7]:
# Open Pickles wich contain a dir of dirs 
# dir_parent: contain all runs dictionary. Labeled by date and time
# dir_son: contain a dictionary of one run. labeled by week (1 - 52). 0 contain hyperparameter information.
def open_pickle():
    tame = {}
    for ld in files:
        with open(f"Pickles//K_one_w//{ld}", 'rb') as f:
            tame[ld] = pickle.load(f)
    return tame

In [8]:
pkls = open_pickle()

In [12]:
pkls.keys()

dict_keys(['08_04_2021_21_29_44', '08_04_2021_21_31_39', '08_04_2021_21_33_43', '08_04_2021_21_35_41', '08_04_2021_21_37_40', '08_04_2021_21_40_03', '08_04_2021_21_41_46'])

In [26]:
d = '08_04_2021_21_41_46'
mape = pkls[d][1]["error"].mean()*100
pkls[d][1][["real", "predict"]].iplot(title = f"MAPE: {mape:.2f}")

In [213]:
pkls['07_04_2021_13_19_55'][0]

{'kernel': 'rbf',
 'C': 4.228517846555483,
 'epsilon': 7.20604168948716,
 'mu': 0.7423118951773942,
 'lmbda': 1.5186395374328805,
 'gamma': 0.00011142604425275418}

In [227]:
# Dataset's last week [1] prediction for hyperparameters on [0]
ma = pkls['07_04_2021_13_19_55'][1].error.mean()*100
pkls['07_04_2021_13_19_55'][1][["real", "predict"]].iplot(title = f"MAPE: {ma:.2f}")

In [168]:
# Join each dir_son sequentially by date. Dir labeled by date.
# recives dir of dirs. Returns dir of ordered 52 weeks prediction
def join_pickle(pkls):
    li_n = {}
    for a in pkls.keys():
        li_n[a] = pd.DataFrame()
        for i in range(1, 53):
            li_n[a] = pd.concat([li_n[a], pkls[a][i]], axis = 0)
        li_n[a] = li_n[a].sort_index()
    return li_n

In [191]:
jointed = join_pickle(pkls)

In [220]:
pkls['07_04_2021_13_19_55'][0]

{'kernel': 'rbf',
 'C': 4.228517846555483,
 'epsilon': 7.20604168948716,
 'mu': 0.7423118951773942,
 'lmbda': 1.5186395374328805,
 'gamma': 0.00011142604425275418}

2.570841558544486

In [226]:
mape = jointed['07_04_2021_13_19_55'].error.mean()*100
jointed['07_04_2021_13_19_55'][["real", "predict"]].iplot(title = f"MAPE: {mape:.2f}")

In [182]:
# concatenate real value and prediction for all runs.
# argument: dir of jointed dataframes. returns real values and all predictions 
def concat_pred(j_p):
    pred_tot = pd.DataFrame()
    pred_tot["real"] = j_p[[*j_p.keys()][0]]['real']
    name = ['real']
    for a in j_p.keys():
        pred_tot = pd.concat([pred_tot, j_p[a]['predict']], axis = 1)
        name.append(a)
    pred_tot.columns = name
    return pred_tot

In [192]:
lok = concat_pred(jointed)

In [193]:
lok.iplot()

In [202]:
# concatenate real value and prediction for all runs.
# argument: dir of jointed dataframes. 
# return: error given by abs((y - pred)/y)
def concat_error(j_p):
    error_tot = pd.DataFrame()
    for a in j_p.keys():
        error_tot = pd.concat([error_tot, j_p[a]['error']], axis = 1)
    error_tot.columns = [*j_p.keys()]
    return error_tot

In [204]:
error_tot = concat_error(jointed)

In [205]:
error_tot.iplot()